In [1]:
#-*- coding: utf-8 -*-
# 라이브러리 import
try:
    from selenium import webdriver
except:
    !pip install selenium
    from selenium import webdriver

try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    !pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager

try:
    from urllib.parse import urlencode
except:
    !pip install urllib3
    from urllib.parse import urlencode
import datetime
import time

try:
    import csv
except:
    !pip install csv
    import csv

In [2]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('start-maximized')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [3]:
# 루프 제한 값
sleepMaxCount = 5
# select query
item_query = "ytd-video-renderer.style-scope.ytd-item-section-renderer"
title_query = "div#title-wrapper yt-formatted-string.style-scope.ytd-video-renderer"
contents_query = "yt-formatted-string[id=description-text]"
like_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
time_query = "ytd-video-meta-block.style-scope.ytd-video-renderer.byline-separated > div#metadata > div#metadata-line > span"
channel_query = "ytd-channel-name#channel-name.long-byline,style-scope,ytd-video-renderer > div#container > div#text-container > yt-formatted-string.style-scope.ytd-channel-name > a"

In [8]:
# 데이터 추출 함수
def startSearch() :
    encode_type = "utf-8"

    chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        search_txt = input("search : ")
        sq={"search_query":search_txt}
        search_encode = urlencode(sq)
        chrome.get("https://www.youtube.com/results?" + search_encode + "&sp=CAISAggF")
        
        i=0
        dataRuw=0
        isEnd=0
        time.sleep(1)

        # 초기 파일 생성
        now_date = datetime.datetime.now()
        cvsFileName = "youtube_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
        with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
            colnames = ['number', 'title', 'contents', 'like', 'time', "channel"]
            w = csv.writer(f)
            w.writerow(colnames)

        # 데이터 추출을 위한 루프
        while True:
            isEnd = 0
            items = chrome.find_elements_by_css_selector(item_query)
            item_len = len(items)

            # 데이터 로드 될때까지 루프, 최대 sleepMaxCount 까지만 루프
            while item_len <= dataRuw and isEnd < sleepMaxCount:
                time.sleep(1)
                items = chrome.find_elements_by_css_selector(item_query)
                item_len = len(items)
                isEnd = isEnd + 1

            # 최대 sleepMaxCount 루프까지 진행 된게 아이라면 isEnd 초기화
            if item_len <= dataRuw:
                break
            else :
                isEnd = 0
                
            with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                w = csv.writer(f)
                number = 0
                for item in items[dataRuw:]:
                    number=number+1
                    title_txt = ""
                    contents_txt = ""
                    like_txt = ""
                    time_txt = ""
                    channel_txt = ""

                    try:
                        title_txt = item.find_element_by_css_selector(title_query).text
                    except Exception as e:
                        print(e)
                        title_txt = ""

                    try:
                        contents_txt = item.find_element_by_css_selector(contents_query).text
                    except Exception as e:
                        print(e)
                        contents_txt = ""

                    try:
                        like_txt = item.find_elements_by_css_selector(like_query)[0].text
                    except Exception as e:
                        print(e)
                        like_txt = ""

                    try:
                        time_txt = item.find_elements_by_css_selector(time_query)[1].text
                    except Exception as e:
                        print(e)
                        time_txt = ""

                    try:
                        channel_txt = item.find_element_by_css_selector(channel_query).text
                    except Exception as e:
                        print(e)
                        channel_txt = ""

                    data = [number, title_txt, contents_txt, like_txt, time_txt, channel_txt]
                    print(data)
                    w.writerow(data)

            dataRuw = item_len

            i = i+1
            print("스크롤 : ", i)
            print("데이터 건수 : ", dataRuw)

            # 브라우저 스크롤 맨 아래로 이동
            chrome.execute_script("window.scrollTo(0, document.getElementById('content').scrollHeight);")
            time.sleep(1)
    except Exception as e:
        print(e)
    finally:
        if chrome:
            chrome.close()


In [9]:
startSearch()

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\gbible\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


search :  복권당첨


[1, '[로또 1등에 당첨된다 명상]돈을 끌어오는명상 💰 로또 1등 당첨되는 명상 💰 부자가 되는 명상 💰 잠재의식 💰 건강과 돈', '... 말해봐, 로또1등당첨되는명상, 돈을끌어오는명상, 부자가되는명상, 복권당첨명상, 로또1등당첨명상, 행운의주파수, 잠재의식 명상, ...', '조회수 없음', '1시간 전', '건강과 돈']
[2, '복권당첨되게해주세요 마이야히 #shorts', '', '조회수 없음', '2시간 전', '여기 구독하면 복권1등 당첨']
[3, '[즉석복권] Korea Lottery 56회 스피또 1000 집에서 강원랜드 분위기를 느껴면서 5억 당첨 가자', '제발 영상 끝까지 시청 부탁드리겠습니다 좋아요 댓글 구독 버튼도 아무거나 한개씩 부탁도 드리겠습니다 시청해 주셔서 감사합니다.', '조회수 4회', '3시간 전', '클릭리치']
[4, '해피투게더 5월 1주차 공동복권 당첨결과', '해피투게더 5월 1주차 공동복권 당첨결과.', '조회수 8회', '4시간 전', 'Happy Together 공식[해피TV]']
[5, '21.05.10 Instant Lottery tickets 즉석복권 스피또1000 10장 복권당첨', '연예인꿈#복권긁기#스피또2000.', '조회수 52회', '7시간 전', '복권당첨']
[6, '[로또 1등에 당첨된다 명상]돈을 끌어오는명상 💰 로또 1등 당첨되는 명상 💰 부자가 되는 명상 💰 잠재의식 💰 소원을 말해봐@UCqhucBevmOzoCdGkqGwWlIw', '... 말해봐, 로또1등당첨되는명상, 돈을끌어오는명상, 부자가되는명상, 복권당첨명상, 로또1등당첨명상, 행운의주파수, 잠재의식 명상, ...', '조회수 92회', '13시간 전', '소원을 말해봐']
[7, '963회 고정6 5수 자료, 단오주로또', '신뢰가는 자료입니다. 잘 선택하셔서 상박하세요~ # #로또 #960 #로또960 #동행복권 #1등당첨번호 #당첨번호 #예상수 #고정수 ...', '조회수 2.5천회', '20시간 전', '단오주로

KeyboardInterrupt: 